### Lineup Optimisation with Genetic Algorithms

In [1]:
import os
import glob
import random
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from sklearn.metrics import mean_squared_error

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
#Concatenate all csv files under a directory
def csv_concatenate(folder_path):
    files = glob.glob(folder_path + "/*.csv")
    df_list = []
    for file in tqdm(files):
        df_list.append(pd.read_csv(file, parse_dates=True, infer_datetime_format=True))
    #Fill nan with 0s as some values are empty for percentage points
    df = pd.concat(df_list).fillna(0).reset_index(drop=True)
    return df

In [4]:
def calculate_MAE(pred, true):
    n = len(pred)
    abs_error = 0 
    for i in range(n):
        abs_error += abs(pred[i] - true[i])
    mae = abs_error/n
    return mae

In [5]:
def calculate_RMSE(pred, true):
    return np.sqrt(mean_squared_error(pred, true))

In [6]:
def calculate_FPTS(df):
    #Scoring rules based on https://www.draftkings.co.uk/help/rules/4
    multipliers = {'PTS':1, '3P': 0.5, 'TRB':1.25, 'AST':1.5, 'STL':2, 'BLK':2, 'TOV':-0.5}
    
    indices = len(df)
    fpts_list = []
    
    for i in tqdm(range(indices)):
        fpts = 0
        doubles = 0
        for stat, multiplier in multipliers.items():
            if stat in ['PTS', 'TRB', 'AST', 'STL', 'BLK']:
                if df.loc[i, stat] >= 10:
                    doubles += 1
            fpts += df.loc[i, stat]*multiplier
        if doubles >= 2:
            fpts += 1.5
        if doubles >= 3:
            fpts += 3
        fpts_list.append(fpts) 
        
    return fpts_list

In [7]:
def add_positions(df):
    
    pos = {'PG':[], 'SG':[], 'SF':[], 'PF':[], 'C':[], 'G':[], 'F':[]}
    
    for i in range(df.shape[0]):
        for key in pos.keys():
            if key in df.loc[i, 'Pos']:
                pos[key].append(1)
            else:
                pos[key].append(0)
    
    for key in pos.keys():
        df[key] = pos[key]

### Genetic Algorithms

In [11]:
#return a list of lineups with indices in df_target
def create_random_lineups(df, num_lineups):
    lineups = {'PG':[], 'SG':[], 'SF':[], 'PF':[], 'C':[], 'G':[], 'F':[], 'Util':[]}
    n = df.shape[0]
    
    for i in range(num_lineups):
        for key in list(lineups.keys())[:7]:
            lineups[key].append(df[df[key]==1].sample(1).index[0])
        
        lineups['Util'].append(df.sample(1).index[0])
    
    df_lineups = pd.DataFrame(lineups).loc[:,['PG', 'SG', 'SF', 'PF', 'C', 'G', 'F', 'Util']]
    
    return df_lineups 

In [12]:
def calculate_fitness(df_lineups):
    fitness = []
    
    for i in range(df_lineups.shape[0]):
        
        salary = df_target.loc[df_lineups.loc[i,:].values, 'Salary'].sum()
        total_FPTS = df_target.loc[df_lineups.loc[i,:].values, 'Pred'].sum()
        
        #Check for duplicates
        if len(set(df_lineups.loc[i,:].values)) < 8:
            fitness.append(0)
        
        #Check for Salary Caps
        elif salary >= salary_cap:
            fitness.append(0)
            
        #Calculate the cumulative predicted FPTS
        else:
            fitness.append(total_FPTS)
        
    return fitness

In [13]:
def breed(df_lineups):
    
    df_parents = df_lineups
    positions = list(df_lineups.columns)[:-1]
    df_children = pd.DataFrame([], columns=positions)
    
    df_fit = df_lineups.sort_values(by='Fitness', ascending=False).reset_index(drop=True)
    parents = df_fit.head(2)

    #Prevent breeding betwee two same lineups
    if all(parents.loc[0,:] != parents.loc[1,:]):
        next_index = 2

        while all(parents.loc[0,:] == parents.loc[1,:]):
            rows = [0, next_index]
            parents = df_fit.loc[rows]
            parents = parents.reset_index(drop=True)
            next_index += 1
            
    parents = parents.loc[:, positions]

    for pos in positions:
        #Add parents and swap each items position by position, creating 16 lineups
        df_children = df_children.append(parents, ignore_index=True)
        df_children.loc[df_children.shape[0]-2, pos] = parents.loc[1, pos]
        df_children.loc[df_children.shape[0]-1, pos] = parents.loc[0, pos]
    
    #Add parents for a total of 18 lineups
    df_children = df_children.append(parents).reset_index(drop=True)
    return df_children

In [14]:
def mutate(df_lineups, df_original, num_mutations):
    positions = list(df_lineups.columns)[:-1]
    df_mutants = pd.DataFrame([], columns=positions)
    df_parent = df_lineups.sort_values(by='Fitness', ascending=False).head(1).loc[:,positions]
    
    #Append the original
    df_mutants = df_mutants.append(df_parent)
    
    #Append the original and then mutate it by replacing it with a random sample for a random position
    for i in range(num_mutations):
        pos_to_swap = random.choice(positions)
        mutant_index = df_original.sample(1).index[0]
        
        df_mutants = df_mutants.append(df_parent, ignore_index=True)
        current_index = df_mutants.shape[0]-1
        df_mutants.loc[current_index, pos_to_swap] = df_original.loc[mutant_index, pos_to_swap]
        
    return df_mutants

In [15]:
def evolution(df_random_lineups, num_mutations, num_generations):
    
    df_init = df_random_lineups

    for i in range(num_generations):
    
        df_children = breed(df_init)
        df_children['Fitness'] = calculate_fitness(df_children)
        
        df_mutants = mutate(df_children, df_random_lineups, num_mutations)
        df_mutants['Fitness'] = calculate_fitness(df_mutants)
        
        df_init = df_mutants
    
    df_init['Fitness'] = calculate_fitness(df_init.drop('Fitness', axis=1))
    df_final = df_init.sort_values(by='Fitness', ascending=False).reset_index(drop=True)
    
    return df_final

In [16]:
def compounding_evolution(population_size=200, num_mutations=50, num_generations=20, num_compounding=5):

    optimal_lineups = []
    
    for i in tqdm(range(num_compounding)):
        df_random_lineups = create_random_lineups(df_target, population_size)
        df_random_lineups['Fitness'] = calculate_fitness(df_random_lineups)
        
        optimal = evolution(df_random_lineups, num_mutations, num_generations)
        
        optimal_lineups.append(list(optimal.loc[0,:].values[:-1].astype(int)))
        
    return optimal_lineups

In [18]:
cwd = os.getcwd().replace('/notebooks','')
data_dir = os.path.join(cwd, 'data')
np.random.seed(8)
salary_cap = 50000

df_pred = pd.read_csv(os.path.join(data_dir, 'Predictions', '20180525.csv'))

add_positions(df_pred)

In [19]:
df_cashline = pd.read_csv(os.path.join(data_dir, 'Contests', 'cashline.csv'))
df_cashline.head(5)

,Date,Cashline,Fee,Entries
0,20180301,255.75,50,136
1,20180302,284.75,100,77
2,20180303,286.25,5,3448
3,20180304,248.50,25,919
4,20180305,254.75,5,1149


In [21]:
date = []
pred = []
actual = []
cashline = []
win = []
earnings = []


df_cashline = pd.read_csv(os.path.join(data_dir, 'Contests', 'cashline.csv'))
df_cashline.head(5)

for i in tqdm(range(df_cashline.shape[0])):
    df_target = df_pred.loc[(df_pred['Salary']!=0)&(df_pred['Date']==df_cashline.loc[i, 'Date'])].reset_index(drop=True)
    
    optimal_lineups = compounding_evolution(population_size=200, num_mutations=50,
                                            num_generations=100, num_compounding=5)
    
    top_lineups = {'Lineup':[], 'Pred':[], 'Actual':[]}

    for j, indices in enumerate(optimal_lineups):
        top_lineups['Lineup'].append(j)
        top_lineups['Pred'].append(df_target.loc[indices, 'Pred'].sum())
        top_lineups['Actual'].append(df_target.loc[indices, 'FPTS'].sum())


    df_lineups = pd.DataFrame(top_lineups).sort_values(by='Pred', ascending=False).reset_index(drop=True)
    df_lineups = df_lineups.loc[:, ['Lineup', 'Pred', 'Actual']]

    rows = optimal_lineups[df_lineups.loc[0, 'Lineup']]
    df_best = df_target.loc[rows,:]
    display(df_best.loc[:,['Date', 'Name', 'Team', 'Pos', 'FPTS', 'Pred', 'Salary']])
    print('Salary:', df_target.loc[rows,'Salary'].sum())
    print('Predicted:', df_target.loc[rows,'Pred'].sum())
    print('Actual:', df_target.loc[rows,'FPTS'].sum())
    print('Cashline:', df_cashline.loc[i, 'Cashline'])
    
    pred.append(df_target.loc[rows,'Pred'].sum())
    actual.append(df_target.loc[rows,'FPTS'].sum())
    cashline.append(df_cashline.loc[i, 'Cashline'])
    
    if df_target.loc[rows, 'FPTS'].sum() >= df_cashline.loc[i, 'Cashline']:
        print('Win:', df_cashline.loc[i, 'Fee'])
        win.append(1)
        earnings.append(df_cashline.loc[i, 'Fee'])
    else:
        print('Lose:', df_cashline.loc[i, 'Fee'])
        win.append(0)
        earnings.append(-df_cashline.loc[i, 'Fee'])
    

HBox(children=(IntProgress(value=0, max=30), HTML(value='')))

HBox(children=(IntProgress(value=0, max=5), HTML(value='')))

,Date,Name,Team,Pos,FPTS,Pred,Salary
14,20180301,George Hill,CLE,PG/SG,16.75,26.510393,4800.0
27,20180301,Kentavious Caldwell-Pope,LAL,SG,24.50,29.972765,6200.0
16,20180301,LeBron James,CLE,SF/PF,54.75,57.830902,11900.0
4,20180301,DeMarre Carroll,BRK,SF/PF,44.50,29.701536,5900.0
9,20180301,Jarrett Allen,BRK,PF/C,35.75,27.110792,4900.0
82,20180301,Bogdan Bogdanovic,SAC,SG/SF,40.00,29.450680,5800.0
44,20180301,Kelly Olynyk,MIA,PF/C,6.75,26.024588,4800.0
81,20180301,De'Aaron Fox,SAC,PG,33.00,29.774086,5600.0


Salary: 49900.0
Predicted: 256.375742
Actual: 256.0
Cashline: 255.75
Win: 50


HBox(children=(IntProgress(value=0, max=5), HTML(value='')))

...

,Date,Name,Team,Pos,FPTS,Pred,Salary
85,20180329,De'Aaron Fox,SAC,PG,19.00,27.176441,4800.0
16,20180329,Stanley Johnson,DET,SG/SF,25.75,20.296760,3600.0
29,20180329,Kevin Durant,GSW,SF/PF,21.25,51.756283,10800.0
107,20180329,LaMarcus Aldridge,SAS,PF/C,45.75,46.140205,9000.0
69,20180329,John Henson,MIL,C,22.75,25.187600,5000.0
12,20180329,Reggie Jackson,DET,PG,25.50,26.298560,4800.0
95,20180329,Skal Labissiere,SAC,PF/C,7.25,29.047941,5200.0
55,20180329,James Johnson,MIA,PF/C,18.00,33.859383,6700.0


Salary: 49900.0
Predicted: 259.763173
Actual: 185.25
Cashline: 267.0
Lose: 5


HBox(children=(IntProgress(value=0, max=5), HTML(value='')))

,Date,Name,Team,Pos,FPTS,Pred,Salary
164,20180330,Tyler Ulis,PHO,PG,44.00,26.546413,4700.0
188,20180330,Joe Ingles,UTA,SG/SF,41.25,33.174590,6100.0
80,20180330,Brandon Ingram,LAL,SG/SF,23.25,32.981483,5600.0
22,20180330,LeBron James,CLE,SF/PF,57.75,61.507526,11800.0
182,20180330,Jusuf Nurkic,POR,C,53.00,33.720900,6500.0
13,20180330,David Nwaba,CHI,SG/SF,6.75,25.321575,4500.0
79,20180330,Kyle Kuzma,LAL,SF/PF,56.00,37.805160,7200.0
64,20180330,Tyrone Wallace,LAC,PG/SG,21.75,22.154203,3400.0


Salary: 49800.0
Predicted: 273.21185
Actual: 303.75
Cashline: 255.0
Win: 25



In [22]:
print(sum(win)/len(win))
print(sum(earnings))

0.43333333333333335
315


In [ ]:
date = list(set(df_cashline['Date'].values))
df_result = pd.DataFrame({'Date': date,
                          'Predicted':pred,
                          'Actual': actual,
                          'Cashline': cashline,
                          'Win':win,
                          'Earnings': earnings
                         })

df_result.loc[:,['Date', 'Predicted', 'Actual', 'Cashline', 'Win', 'Earnings']]

### Baseline - random 10,000

In [ ]:
df_result.loc[:,['Date', 'Predicted', 'Actual', 'Cashline', 'Win', 'Earnings']]

In [ ]:
df_random_lineups = create_random_lineups(df_target, 200)
df_random_lineups['Fitness'] = calculate_fitness(df_random_lineups)
df_random_lineups.sort_values(by='Fitness', ascending=False).head(10)

### Visualisation

In [ ]:
df_baseline = csv_concatenate(os.path.join(data_dir, data_dir, 'Dataframes', 'modelling', 'baseline'))
df_baseline['Baseline'] = calculate_FPTS(df_baseline)

In [ ]:
#target_date = 20180310
target_date = 20180326

In [ ]:
df_target = df_pred.loc[(df_pred['Salary']!=0)&(df_pred['Date']==target_date)].reset_index(drop=True)
    
optimal_lineups = compounding_evolution(population_size=200, num_mutations=50,
                                        num_generations=100, num_compounding=5)

top_lineups = {'Lineup':[], 'Pred':[], 'Actual':[]}

for j, indices in enumerate(optimal_lineups):
    top_lineups['Lineup'].append(j)
    top_lineups['Pred'].append(df_target.loc[indices, 'Pred'].sum())
    top_lineups['Actual'].append(df_target.loc[indices, 'FPTS'].sum())


df_lineups = pd.DataFrame(top_lineups).sort_values(by='Pred', ascending=False).reset_index(drop=True)
df_lineups = df_lineups.loc[:, ['Lineup', 'Pred', 'Actual']]

rows = optimal_lineups[df_lineups.loc[0, 'Lineup']]
df_best = df_target.loc[rows,:]
display(df_best.loc[:,['Date', 'Name', 'Team', 'Pos', 'FPTS', 'Pred', 'Salary']])
print('Salary:', df_target.loc[rows,'Salary'].sum())
print('Predicted:', df_target.loc[rows,'Pred'].sum())
print('Actual:', df_target.loc[rows,'FPTS'].sum())
print('Cashline:', int(df_cashline.loc[df_cashline['Date']==target_date, 'Cashline'].values))

In [ ]:
print(calculate_MAE(df_best['FPTS'].values, df_best['Pred'].values))
print(calculate_RMSE(df_best['FPTS'], df_best['Pred']))

In [ ]:
df_best

In [ ]:
Baseline = []

for i in range(df_best.shape[0]):
    df_best = df_best.reset_index(drop=True)
    Baseline.append(df_baseline.loc[(df_baseline['Name']==df_best.loc[i, 'Name'])&(df_baseline['Date']==df_best.loc[i,'Date']), 'Baseline'])

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go

names = [name.split(' ')[0][:1] + '. '+ name.split(' ')[1] for name in df_best['Name']]
positions = ['(PG)', '(SG)', '(SF)','(PF)','(C)','(G)','(F)','(Utility)']

names = [names[i]+'\n'+positions[i] for i in range(8)]

trace1 = go.Bar(
    x=names,
    y=df_best['FPTS'],
    name='Actual FPTS'
)

trace2 = go.Bar(
    x=names,
    y=df_best['Pred'],
    name='Prediction'
)

trace3 = go.Bar(
    x=names,
    y=Baseline,
    name='Baseline'
)


data = [trace1, trace2, trace3]
layout = go.Layout(
    
)

layout = go.Layout(
        title = 'FPTS: Actual and Predicted',
        barmode='group',
        legend = {"x":0.85, 'y':0.95, 'borderwidth': 1},
        yaxis = {"title":"FPTS"},
    )

config={'showLink': False}

fig = go.Figure(data=data, layout=layout)
plot_url = py.plot(fig, filename='prediction0326')
py.iplot(fig, filename='prediction0326')